In [0]:
df_silver_clean=spark.read.format("delta").load("/Volumes/api/api/processed_files/silver/")

In [0]:
df_gold_transparency = (
    df_silver_clean.groupBy("is_trans")
    .count()
    .withColumnRenamed("count", "color_count")
)


In [0]:
from pyspark.sql.functions import *

df_gold_external = df_silver_clean.select(
    "id",
    when(col("bricklink_id").isNotNull(), 1).otherwise(0).alias("has_bricklink"),
    when(col("brickowl_id").isNotNull(), 1).otherwise(0).alias("has_brickowl"),
    when(col("ldraw_id").isNotNull(), 1).otherwise(0).alias("has_ldraw"),
    when(col("lego_id").isNotNull(), 1).otherwise(0).alias("has_lego"),
    when(col("peeron_id").isNotNull(), 1).otherwise(0).alias("has_peeron")
)

df_gold_external_summary = df_gold_external.agg(
    sum("has_bricklink").alias("bricklink_mapped"),
    sum("has_brickowl").alias("brickowl_mapped"),
    sum("has_ldraw").alias("ldraw_mapped"),
    sum("has_lego").alias("lego_mapped"),
    sum("has_peeron").alias("peeron_mapped")
)


In [0]:

df_gold_transparency.write.format("delta").mode("overwrite").save("/Volumes/api/api/processed_files/gold/transparency_summary")
df_gold_external_summary.write.format("delta").mode("overwrite").save("/Volumes/api/api/processed_files/gold/external_mapping_summary")

In [0]:
%sql
Select * from delta . `/Volumes/api/api/processed_files/gold/external_mapping_summary/`

In [0]:
%sql
select * from delta. `/Volumes/api/api/processed_files/gold/transparency_summary/`